<a href="https://colab.research.google.com/github/exem-ai-isoh/Hands-On-Machine-Learning/blob/master/support_vector_machines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 서포트 벡터의 아이디어

서포트 벡터 머신의 근본적인 아이디어는 클래스 사이에 가능한 한 넓은 '도로'를 내는 것입니다. 다시 말해 두 클래스를 구분하는 결정 경계와 샘플 사이의 마진을 가능한 한 크게 하는 것이 목적입니다. 소프트 마진 분류를 할 때 SVM은 두 클래스를 완전히 나누는 것과 가장 넓은 도로를 내는 것 사이에서 절충안을 찾습니다. 모든 샘플이 올바르게 분류되어 있다면 하드 마진 분류라고 합니다. 하드마진분류는 이상치에 민감하게 반응합니다.  

SVM 회귀의 경우 도로 안에 가능한 한 많은 샘플이 들어가도록 학습합니다. 도로의 폭은 하이퍼파라미터로 조절합니다.

- 서포트 벡터란?

서포트 벡터는 SVM이 훈련된 후에 경계를 포함해 도로에 놓인 어떤 샘플입니다. 결정경계는 서포트 벡터만이 영향을 줄 수 있으며 서포트 벡터가 아닌 샘플은 결정경계에 영향을 줄 수 없습니다.

- 서포트 벡터 이용시 주의사항

SVM은 가능한 큰 도로를 찾는 것이므로 스케일이 맞지 않을 경우 작은 특성을 무시하는 경향이 있습니다.

In [2]:
# (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20.

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "svm"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

캘리포니아 주택 가격 데이터셋에 SVM 회귀를 훈련시켜보자.

In [3]:
# data load
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
X = housing["data"]
y = housing["target"]

In [6]:
# data split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# scale
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
# train by LinearSVR 
from sklearn.svm import LinearSVR

lin_svr = LinearSVR(random_state=42)
lin_svr.fit(X_train_scaled, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


LinearSVR(random_state=42)

In [9]:
# trainset에서 performance
from sklearn.metrics import mean_squared_error

y_pred = lin_svr.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
mse

0.9641780189948642

In [10]:
# RMSE
np.sqrt(mse)

0.9819256687727764

use randomized search with cross validation to find the appropriate hyperparameter values for C and gamma

In [11]:
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform # reciprocal : 역수

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(SVR(), param_distributions, n_iter=10, verbose=2, cv=3, random_state=42)
rnd_search_cv.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END .....C=4.745401188473625, gamma=0.07969454818643928; total time=  13.1s
[CV] END .....C=4.745401188473625, gamma=0.07969454818643928; total time=  10.4s
[CV] END .....C=4.745401188473625, gamma=0.07969454818643928; total time=  11.6s
[CV] END .....C=8.31993941811405, gamma=0.015751320499779724; total time=  10.1s
[CV] END .....C=8.31993941811405, gamma=0.015751320499779724; total time=  13.3s
[CV] END .....C=8.31993941811405, gamma=0.015751320499779724; total time=  10.1s
[CV] END ....C=2.560186404424365, gamma=0.002051110418843397; total time=   9.6s
[CV] END ....C=2.560186404424365, gamma=0.002051110418843397; total time=  13.9s
[CV] END ....C=2.560186404424365, gamma=0.002051110418843397; total time=  12.9s
[CV] END ....C=1.5808361216819946, gamma=0.05399484409787431; total time=  10.9s
[CV] END ....C=1.5808361216819946, gamma=0.05399484409787431; total time=  10.0s
[CV] END ....C=1.5808361216819946, gamma=0.05399

RandomizedSearchCV(cv=3, estimator=SVR(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fd818e610d0>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fd818e613d0>},
                   random_state=42, verbose=2)

In [12]:
# best parameter
rnd_search_cv.best_estimator_

SVR(C=4.745401188473625, gamma=0.07969454818643928)

In [13]:
# performance after tuning

y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
np.sqrt(mse)

0.5727524770785356

비선형 분류

- 다항식 커널 : 다항식 특성을 추가한 것.
- 유사도 특성 추가 : 각 샘플의 특정 랜드마크와 닮았는지 측정
- 가우시안 RBF 커널 : 유사도 특성을 많이 추가하는 것과 비슷한 결과를 실제로 특성을 추가하지 않고도 얻을 수 있습니다.

